In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time


In [3]:
url = 'http://www.portalaluguelsocial.rj.gov.br/Default.asp'

In [4]:
#processa a string que contem o Total
def parseTotal(string):
    index = string.find('Total Pgto')
    new_str = string[index:]
    new_str = new_str.split('\xa0')
    new_str = new_str[1].split(',')[0]
    new_str = new_str.replace('.','')
    total = float(new_str)
    return total

#procura pelo Total no HTML da página, processa e retorna como float
def getTotal(content):
    soup = BeautifulSoup(content, 'html.parser')
    destaque = soup.find_all(name='td', attrs={'class':'Destaque'})
    text = str(destaque)
    total = parseTotal(text)
    return total

In [5]:
#testing
req = requests.get(url)
if req.status_code == 200:
    print('Requisição bem sucedida!')
    content = req.content
print(getTotal(content))

Requisição bem sucedida!
0.0


In [6]:
#dataframe para armazenar os dados
df = pd.DataFrame(columns = ['Mês/Ano','Comunidade','Total de Alugueis'])
#df = df.append({'Mês/Ano':'12/2012','Comunidade':'Rocinha','Total de Alugueis':200},ignore_index=True)
df

,Mês/Ano,Comunidade,Total de Alugueis


In [7]:
#cria o webdriver
driver = webdriver.Firefox()
driver.get(url)

#referencia o seletor de datas e pega uma lista com as opções
select_datas = Select(driver.find_element_by_id("selChaveMesAnoPagto"))
datas = select_datas.options


In [8]:
#Como a página é dinâmica, precisamos pegar o tamanho da lista das opções, iterar pelos indíces 
#atualizar a cada iteração (ufa!). Assim, iremos fazer loops aninhados para varrer todas as combinações de
#data e comunidade para montar nosso dataset
start = time.time()
rows = 0
datas_range = range(len(datas))
for i in datas_range:
    #atualiza lista
    select_datas = Select(driver.find_element_by_id("selChaveMesAnoPagto"))
    datas = select_datas.options
    #recebe o valor da data
    data = datas[i].get_attribute("text")
    print("Data: %s" % data)
    
    if data == 'Selecione...':
        continue
        
    datas[i].click()
    
    select_municipios = Select(driver.find_element_by_id("selChaveMunicipio"))
    municipios = select_municipios.options
    municipios_range = len(municipios)
    j=0
    
    while j < municipios_range:
        select_municipios = Select(driver.find_element_by_id("selChaveMunicipio"))
        municipios = select_municipios.options
        municipios_range = len(municipios)
        municipio = municipios[j].get_attribute("text")
        print("Município/Comunidade: %s" % municipio)
        
        if municipio == 'Selecione...':
            j+=1
            continue
            
        municipios[j].click()        
        #referencia o botão
        button = driver.find_element_by_name("btnPesquisar")
        button.click()
        #time.sleep(5)
        html = driver.page_source
        total = getTotal(html)
        
        print("Valor Total Pgto: ", total)
        df = df.append({'Mês/Ano': data,'Comunidade': municipio,'Total de Alugueis':total},ignore_index=True)
        rows+=1
        print("Linhas na tabela: ", rows)
        print()
        j+=1
        
end = time.time()
total_time = end-start
print("Tempo de execução: ",total_time, " segundos")
        

        

Data: Selecione...
Data: 09/2019
Município/Comunidade: Selecione...
Município/Comunidade: ANGRA DOS REIS
Valor Total Pgto:  17000.0
Linhas na tabela:  1

Município/Comunidade: AREAL
Valor Total Pgto:  31200.0
Linhas na tabela:  2

Município/Comunidade: BARRA MANSA
Valor Total Pgto:  400.0
Linhas na tabela:  3

Município/Comunidade: COMUNIDADE ALEMÃO
Valor Total Pgto:  114000.0
Linhas na tabela:  4

Município/Comunidade: COMUNIDADE BARROS FILHO
Valor Total Pgto:  400.0
Linhas na tabela:  5

Município/Comunidade: COMUNIDADE CAJU - SEEDUC
Valor Total Pgto:  40400.0
Linhas na tabela:  6

Município/Comunidade: COMUNIDADE CCPL
Valor Total Pgto:  156800.0
Linhas na tabela:  7

Município/Comunidade: COMUNIDADE CHIQUEIRINHO
Valor Total Pgto:  2800.0
Linhas na tabela:  8

Município/Comunidade: COMUNIDADE CONAB
Valor Total Pgto:  257600.0
Linhas na tabela:  9

Município/Comunidade: COMUNIDADE INDÍGENA
Valor Total Pgto:  800.0
Linhas na tabela:  10

Município/Comunidade: COMUNIDADE ITAOCA
Valor To

Valor Total Pgto:  1000.0
Linhas na tabela:  89

Município/Comunidade: PROCESSOS RESOLUÇÃO SEOBRAS 768
Valor Total Pgto:  1600.0
Linhas na tabela:  90

Município/Comunidade: QUEIMADOS
Valor Total Pgto:  1000.0
Linhas na tabela:  91

Município/Comunidade: RIO DE JANEIRO - JUSTIÇA VALOR BENEF
Valor Total Pgto:  2500.0
Linhas na tabela:  92

Município/Comunidade: RIO DE JANEIRO - PROCESSOS JUDICIAIS
Valor Total Pgto:  22800.0
Linhas na tabela:  93

Município/Comunidade: SÃO GONÇALO
Valor Total Pgto:  11600.0
Linhas na tabela:  94

Município/Comunidade: SÃO JOSÉ DO VALE DO RIO PRETO
Valor Total Pgto:  27600.0
Linhas na tabela:  95

Município/Comunidade: SUMIDOURO
Valor Total Pgto:  16400.0
Linhas na tabela:  96

Município/Comunidade: TERESÓPOLIS
Valor Total Pgto:  169000.0
Linhas na tabela:  97

Data: 06/2019
Município/Comunidade: Selecione...
Município/Comunidade: ANGRA DOS REIS
Valor Total Pgto:  17000.0
Linhas na tabela:  98

Município/Comunidade: AREAL
Valor Total Pgto:  31600.0
Linhas

Valor Total Pgto:  650.0
Linhas na tabela:  176

Município/Comunidade: COMUNIDADE ROCINHA
Valor Total Pgto:  2400.0
Linhas na tabela:  177

Município/Comunidade: COMUNIDADE SANTA MARTHA
Valor Total Pgto:  1600.0
Linhas na tabela:  178

Município/Comunidade: COMUNIDADE SKOL
Valor Total Pgto:  218800.0
Linhas na tabela:  179

Município/Comunidade: COMUNIDADE VILA SÃO PEDRO
Valor Total Pgto:  4000.0
Linhas na tabela:  180

Município/Comunidade: COMUNIDADE VILA UNIÃO
Valor Total Pgto:  2000.0
Linhas na tabela:  181

Município/Comunidade: DUQUE DE CAXIAS
Valor Total Pgto:  9000.0
Linhas na tabela:  182

Município/Comunidade: NITERÓI
Valor Total Pgto:  486000.0
Linhas na tabela:  183

Município/Comunidade: NOVA FRIBURGO
Valor Total Pgto:  19500.0
Linhas na tabela:  184

Município/Comunidade: PETRÓPOLIS
Valor Total Pgto:  354500.0
Linhas na tabela:  185

Município/Comunidade: PROCESSOS JUDICIAIS MUNICÍPIO RJ
Valor Total Pgto:  1000.0
Linhas na tabela:  186

Município/Comunidade: PROCESSOS RES

Valor Total Pgto:  3200.0
Linhas na tabela:  264

Município/Comunidade: COMUNIDADE ALEMÃO
Valor Total Pgto:  114800.0
Linhas na tabela:  265

Município/Comunidade: COMUNIDADE BARROS FILHO
Valor Total Pgto:  400.0
Linhas na tabela:  266

Município/Comunidade: COMUNIDADE CAJU - SEEDUC
Valor Total Pgto:  40800.0
Linhas na tabela:  267

Município/Comunidade: COMUNIDADE CCPL
Valor Total Pgto:  158400.0
Linhas na tabela:  268

Município/Comunidade: COMUNIDADE CHIQUEIRINHO
Valor Total Pgto:  2800.0
Linhas na tabela:  269

Município/Comunidade: COMUNIDADE CONAB
Valor Total Pgto:  262000.0
Linhas na tabela:  270

Município/Comunidade: COMUNIDADE INDÍGENA
Valor Total Pgto:  800.0
Linhas na tabela:  271

Município/Comunidade: COMUNIDADE ITAOCA
Valor Total Pgto:  158000.0
Linhas na tabela:  272

Município/Comunidade: COMUNIDADE JACAREZINHO
Valor Total Pgto:  579200.0
Linhas na tabela:  273

Município/Comunidade: COMUNIDADE PARQUE JOÃO GOULART
Valor Total Pgto:  4800.0
Linhas na tabela:  274

Munic

Valor Total Pgto:  1200.0
Linhas na tabela:  352

Município/Comunidade: QUEIMADOS
Valor Total Pgto:  1000.0
Linhas na tabela:  353

Município/Comunidade: RIO DE JANEIRO - JUSTIÇA VALOR BENEF
Valor Total Pgto:  2250.0
Linhas na tabela:  354

Município/Comunidade: RIO DE JANEIRO - PROCESSOS JUDICIAIS
Valor Total Pgto:  11200.0
Linhas na tabela:  355

Município/Comunidade: SÃO GONÇALO
Valor Total Pgto:  56000.0
Linhas na tabela:  356

Município/Comunidade: SÃO JOSÉ DO VALE DO RIO PRETO
Valor Total Pgto:  28000.0
Linhas na tabela:  357

Município/Comunidade: SUMIDOURO
Valor Total Pgto:  17200.0
Linhas na tabela:  358

Município/Comunidade: TERESÓPOLIS
Valor Total Pgto:  293500.0
Linhas na tabela:  359

Data: 10/2018
Município/Comunidade: Selecione...
Município/Comunidade: ANGRA DOS REIS
Valor Total Pgto:  13500.0
Linhas na tabela:  360

Município/Comunidade: AREAL
Valor Total Pgto:  31600.0
Linhas na tabela:  361

Município/Comunidade: BARRA MANSA
Valor Total Pgto:  400.0
Linhas na tabela:

...

Valor Total Pgto:  298400.0
Linhas na tabela:  2208

Município/Comunidade: COMUNIDADE ITAOCA
Valor Total Pgto:  165600.0
Linhas na tabela:  2209

Município/Comunidade: COMUNIDADE PAVÃOZINHO
Valor Total Pgto:  19200.0
Linhas na tabela:  2210

Município/Comunidade: COMUNIDADE SANTA MARTHA
Valor Total Pgto:  800.0
Linhas na tabela:  2211

Município/Comunidade: COMUNIDADE SKOL
Valor Total Pgto:  232400.0
Linhas na tabela:  2212

Município/Comunidade: COMUNIDADE VILA SÃO PEDRO
Valor Total Pgto:  4000.0
Linhas na tabela:  2213

Município/Comunidade: NITERÓI
Valor Total Pgto:  1274800.0
Linhas na tabela:  2214

Município/Comunidade: NOVA FRIBURGO
Valor Total Pgto:  1348500.0
Linhas na tabela:  2215

Município/Comunidade: PETRÓPOLIS
Valor Total Pgto:  434000.0
Linhas na tabela:  2216

Município/Comunidade: SÃO GONÇALO
Valor Total Pgto:  412800.0
Linhas na tabela:  2217

Município/Comunidade: SÃO JOSÉ DO VALE DO RIO PRETO
Valor Total Pgto:  156000.0
Linhas na tabela:  2218

Município/Comunidade

Município/Comunidade: Selecione...
Município/Comunidade: AREAL
Valor Total Pgto:  121600.0
Linhas na tabela:  2298

Município/Comunidade: BARRA MANSA
Valor Total Pgto:  15200.0
Linhas na tabela:  2299

Município/Comunidade: BOM JARDIM
Valor Total Pgto:  92000.0
Linhas na tabela:  2300

Município/Comunidade: COMUNIDADE ALEMÃO
Valor Total Pgto:  158000.0
Linhas na tabela:  2301

Município/Comunidade: COMUNIDADE CCPL
Valor Total Pgto:  510400.0
Linhas na tabela:  2302

Município/Comunidade: COMUNIDADE CONAB
Valor Total Pgto:  297200.0
Linhas na tabela:  2303

Município/Comunidade: COMUNIDADE ITAOCA
Valor Total Pgto:  312800.0
Linhas na tabela:  2304

Município/Comunidade: COMUNIDADE PAVÃOZINHO
Valor Total Pgto:  20400.0
Linhas na tabela:  2305

Município/Comunidade: COMUNIDADE SANTA MARTHA
Valor Total Pgto:  400.0
Linhas na tabela:  2306

Município/Comunidade: COMUNIDADE SKOL
Valor Total Pgto:  230800.0
Linhas na tabela:  2307

Município/Comunidade: NITERÓI
Valor Total Pgto:  1272400.0
Li

Valor Total Pgto:  152400.0
Linhas na tabela:  2387

Município/Comunidade: COMUNIDADE CCPL
Valor Total Pgto:  502800.0
Linhas na tabela:  2388

Município/Comunidade: COMUNIDADE CONAB
Valor Total Pgto:  302000.0
Linhas na tabela:  2389

Município/Comunidade: COMUNIDADE PAVÃOZINHO
Valor Total Pgto:  4400.0
Linhas na tabela:  2390

Município/Comunidade: COMUNIDADE SANTA MARTHA
Valor Total Pgto:  2000.0
Linhas na tabela:  2391

Município/Comunidade: COMUNIDADE SKOL
Valor Total Pgto:  230000.0
Linhas na tabela:  2392

Município/Comunidade: NITERÓI
Valor Total Pgto:  1298400.0
Linhas na tabela:  2393

Município/Comunidade: NOVA FRIBURGO
Valor Total Pgto:  1401500.0
Linhas na tabela:  2394

Município/Comunidade: PETRÓPOLIS
Valor Total Pgto:  446500.0
Linhas na tabela:  2395

Município/Comunidade: SÃO GONÇALO
Valor Total Pgto:  396800.0
Linhas na tabela:  2396

Município/Comunidade: SÃO JOSÉ DO VALE DO RIO PRETO
Valor Total Pgto:  142400.0
Linhas na tabela:  2397

Município/Comunidade: SAPUCAI

Valor Total Pgto:  130000.0
Linhas na tabela:  2477

Município/Comunidade: COMUNIDADE CCPL
Valor Total Pgto:  456800.0
Linhas na tabela:  2478

Município/Comunidade: COMUNIDADE SKOL
Valor Total Pgto:  236800.0
Linhas na tabela:  2479

Município/Comunidade: NITERÓI
Valor Total Pgto:  1237200.0
Linhas na tabela:  2480

Município/Comunidade: NOVA FRIBURGO
Valor Total Pgto:  1394500.0
Linhas na tabela:  2481

Município/Comunidade: PETRÓPOLIS
Valor Total Pgto:  424000.0
Linhas na tabela:  2482

Município/Comunidade: SÃO GONÇALO
Valor Total Pgto:  459600.0
Linhas na tabela:  2483

Município/Comunidade: SÃO JOSÉ DO VALE DO RIO PRETO
Valor Total Pgto:  138800.0
Linhas na tabela:  2484

Município/Comunidade: SAPUCAIA
Valor Total Pgto:  83200.0
Linhas na tabela:  2485

Município/Comunidade: SUMIDOURO
Valor Total Pgto:  84800.0
Linhas na tabela:  2486

Município/Comunidade: TERESÓPOLIS
Valor Total Pgto:  1331500.0
Linhas na tabela:  2487

Data: 02/2012
Município/Comunidade: Selecione...
Município

Valor Total Pgto:  123200.0
Linhas na tabela:  2566

Município/Comunidade: BOM JARDIM
Valor Total Pgto:  150800.0
Linhas na tabela:  2567

Município/Comunidade: COMUNIDADE ALEMÃO
Valor Total Pgto:  100400.0
Linhas na tabela:  2568

Município/Comunidade: COMUNIDADE CCPL
Valor Total Pgto:  459600.0
Linhas na tabela:  2569

Município/Comunidade: COMUNIDADE SKOL
Valor Total Pgto:  30400.0
Linhas na tabela:  2570

Município/Comunidade: NITERÓI
Valor Total Pgto:  1336800.0
Linhas na tabela:  2571

Município/Comunidade: NOVA FRIBURGO
Valor Total Pgto:  1235000.0
Linhas na tabela:  2572

Município/Comunidade: PETRÓPOLIS
Valor Total Pgto:  431000.0
Linhas na tabela:  2573

Município/Comunidade: SÃO GONÇALO
Valor Total Pgto:  486800.0
Linhas na tabela:  2574

Município/Comunidade: SÃO JOSÉ DO VALE DO RIO PRETO
Valor Total Pgto:  146000.0
Linhas na tabela:  2575

Município/Comunidade: SUMIDOURO
Valor Total Pgto:  83200.0
Linhas na tabela:  2576

Município/Comunidade: TERESÓPOLIS
Valor Total Pgto:

In [9]:
df

,Mês/Ano,Comunidade,Total de Alugueis
0,09/2019,ANGRA DOS REIS,17000.0
1,09/2019,AREAL,31200.0
2,09/2019,BARRA MANSA,400.0
3,09/2019,COMUNIDADE ALEMÃO,114000.0
4,09/2019,COMUNIDADE BARROS FILHO,400.0
5,09/2019,COMUNIDADE CAJU - SEEDUC,40400.0
6,09/2019,COMUNIDADE CCPL,156800.0
7,09/2019,COMUNIDADE CHIQUEIRINHO,2800.0
8,09/2019,COMUNIDADE CONAB,257600.0
9,09/2019,COMUNIDADE INDÍGENA,800.0


In [10]:
df.to_excel("Aluguel_Social_RJ.xlsx")

In [14]:
df.to_csv("Aluguel_Social_RJ",index=False)